In [16]:
!pip install --upgrade albumentations

In [5]:
%cd CAP_augmentation/

/home/ubuntu/workspace/CAP_augmentation


In [6]:
import cv2
import numpy as np
import pprint
from pathlib import Path
from matplotlib import pyplot as plt
from BEV.bev_transform import BEV
from src.cap_aug import CAP_AUG, CAP_AUG_Multiclass, CAP_Albu
from src.utils import show_image, draw_bboxes, show_image_and_masks
import albumentations as A

In [ ]:
show_image(image)

In [101]:
import json

def extractAnnotationId(image_name):
    annotation_id = image_name.split('_')[-1].split('.')[0]
    return annotation_id

def extractFileName(annotation_path, annotation_id, encoding='utf8'):
    with open(annotation_path, encoding=encoding) as cocojson:
        annotations_json = json.load(cocojson)

    annotations = annotations_json['annotations']
    specific_image_id = []
    for annotation in annotations:
        if int(annotation_id) == annotation['id']:
            specific_image_id = annotation['image_id']
            break

    print(specific_image_id)
    images = annotations_json['images']
    filename = ''
    for image in images:
        if image['id']==specific_image_id:
            filename = image['file_name']
            break

    parts = filename.split('_')
    directory = "_".join(parts[:-1])

    image_list = []
    for image in images:
        if directory in image['file_name']:
            image_list.append(image['id'])
    return image_list

def findSimilarAnnotations(annotation_path, file_id_list, encoding='utf8'):
    with open(annotation_path, encoding=encoding) as cocojson:
        annotations_json = json.load(cocojson)
    annotations = annotations_json['annotations']

    annotation_list = []
    for annotation in annotations:
        if annotation['image_id'] in file_id_list:
            annotation_list.append(annotation['id'])

    return annotation_list

annotation_id = extractAnnotationId("crop_annotationId_9026.jpg")
file_name = extractFileName("/home/ubuntu/workspace/datasets/final_dataset/train/annotations/EO-train.json", annotation_id)
ann_list= findSimilarAnnotations("/home/ubuntu/workspace/datasets/final_dataset/train/annotations/EO-train.json", file_name)
print(ann_list)

7868
[7056, 7057, 7058, 7059, 7060, 7061, 7062, 7063, 7064, 7065, 7066, 7067, 7068, 7069, 7070, 7071, 7072, 7073, 7074, 7075, 7076, 7077, 7078, 7079, 7080, 7081, 7082, 7083, 7084, 7085, 7086, 7087, 7088, 7089, 7090, 7091, 7092, 7093, 7094, 7095, 7096, 7097, 7098, 7099, 7100, 7101, 7102, 7103, 7104, 7105, 7106, 7107, 7108, 7109, 7110, 7111, 7112, 7113, 7114, 7115, 7116, 7117, 7118, 7119, 7120, 7121, 7122, 7123, 7124, 7125, 7126, 7127, 7128, 7129, 7130, 7131, 7132, 7133, 7134, 7135, 7136, 7137, 7138, 7139, 7140, 7141, 7142, 7143, 7144, 7145, 7146, 7147, 7148, 7149, 7150, 7151, 7152, 7153, 7154, 7155, 7156, 7157, 7158, 7159, 7160, 7161, 7162, 7163, 7164, 7165, 7166, 7167, 7168, 7169, 7170, 7171, 7172, 7173, 7174, 7175, 7176, 7177, 7178, 7179, 7180, 7181, 7182, 7183, 7184, 7185, 7186, 7187, 7188, 7189, 7190, 7191, 7192, 7193, 7194, 7195, 7196, 7197, 7198, 7199, 7200, 7201, 7202, 7203, 7204, 7205, 7206, 7207, 7208, 7209, 7210, 7211, 7212, 7213, 7214, 7215, 7216, 7217, 7218, 7219, 7220, 7221

In [7]:
HUMAN_DATASET_ROOT = Path('/home/ubuntu/workspace/visualization/crop/selected_crops')
SOURCE_IMAGES = sorted(list(HUMAN_DATASET_ROOT.glob('*.jpg')))

In [107]:
object = cv2.imread(str(SOURCE_IMAGES[5]))
print(object.shape)

(18, 25, 3)


In [11]:
cap_aug_pixels = CAP_AUG(SOURCE_IMAGES,  bev_transform=None, 
                                               n_objects_range=[1,4], 
                                               h_range=[10,30],
                                               x_range=[100, 700],
                                               y_range=[100,500],
                                               coords_format='xyxy'
                                                )

In [14]:
DEST_DATASET_ROOT = Path('/home/ubuntu/workspace/visualization/crop/google_bg')
DEST_IMAGES = sorted(list(DEST_DATASET_ROOT.glob('*.png')))

bbox_list = [] 
i = 0


while i < 5001:
    for img in DEST_IMAGES:
        image = cv2.imread(str(img))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result_image, bbox_coord, semantic_mask, instance_mask = cap_aug_pixels(image)

        # if len(bbox_coord) == 0:
        #      print(bbox_coord)
        #      continue
    
        output_dir = '/home/ubuntu/workspace/visualization/augmented_data/'
        output_name = str(i) + '.jpg'
        output_path = output_dir + output_name

        # print(instance_mask)
        
        cv2.imwrite(output_path, result_image)
        
        bboxes = bbox_coord.tolist()
        temp = [output_name, bboxes]
        bbox_list.append(temp)
        i += 1


# # Draw bounding boxes
# result_image_vis, result_mask_vis = draw_bboxes(result_image, bbox_coord, mask=instance_mask)

# show_image_and_masks(image, result_image, result_mask_vis, result_image_vis)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [38]:
import json
from PIL import Image

def create_coco_json(image_folder, bbox_list, output_path):
    coco_data = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Add categories if needed (e.g., person, car, etc.)
    # coco_data["categories"] = [
    #     {"id": 1, "name": "person", "supercategory": "object"},
    #     {"id": 2, "name": "car", "supercategory": "object"},
    #     ...
    # ]

    image_id = 0
    annotation_id = 0
    
    for image_filename, bboxes in bbox_list:
        image_path = f"{image_folder}/{image_filename}"

        image = Image.open(image_path)

        # Create COCO image entry
        image_data = {
            "id": image_id,
            "file_name": image_filename,
            "width": image.width,
            "height": image.height
        }
        coco_data["images"].append(image_data)

        for bbox in bboxes:
            # Create COCO annotation entry
            annotation_data = {
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 0,  # Replace with appropriate category ID
                "bbox": bbox,  # Format: [x, y, width, height]
                "area": bbox[2] * bbox[3]
            }
            coco_data["annotations"].append(annotation_data)

            annotation_id += 1
        
        image_id += 1

    category_data = {
        "id" : 0,
        "name" : "drone",
        "supercategory" : "none"
    }

    coco_data["categories"].append(category_data)

    # Save COCO JSON file
    with open(output_path, "w") as f:
        json.dump(coco_data, f)

print(bbox_list)
# Example usage
image_folder = "/home/ubuntu/workspace/visualization/augmented_data/image"
output_path = "/home/ubuntu/workspace/visualization/augmented_data/train.json"

create_coco_json(image_folder, bbox_list, output_path)

[['0.jpg', [[135, 322, 12, 10], [567, 467, 12, 10], [437, 260, 28, 24], [316, 128, 22, 17]]], ['1.jpg', [[169, 189, 31, 23], [443, 382, 18, 14]]], ['2.jpg', [[327, 111, 52, 29], [502, 447, 34, 24], [537, 94, 47, 28], [665, 473, 28, 15]]], ['3.jpg', [[359, 130, 22, 18], [666, 127, 30, 18], [263, 240, 22, 20], [444, 378, 16, 13]]], ['4.jpg', [[433, 98, 26, 24], [274, 419, 13, 10]]], ['5.jpg', [[684, 344, 14, 12], [107, 363, 25, 23], [401, 191, 31, 22]]], ['6.jpg', [[651, 245, 18, 13], [412, 142, 31, 27], [213, 452, 25, 23], [495, 376, 105, 26]]], ['7.jpg', [[302, 176, 38, 27], [481, 100, 18, 15]]], ['8.jpg', [[110, 336, 36, 27], [214, 344, 16, 13], [328, 295, 35, 21]]], ['9.jpg', [[340, 441, 35, 15]]], ['10.jpg', [[269, 315, 45, 28]]], ['11.jpg', [[360, 105, 16, 10], [631, 108, 13, 10], [254, 451, 38, 18]]], ['12.jpg', [[340, 241, 30, 19], [299, 161, 13, 11], [692, 117, 15, 13], [242, 174, 17, 13]]], ['13.jpg', [[511, 350, 62, 29], [224, 204, 37, 29]]], ['14.jpg', [[169, 203, 16, 10]]], 

In [210]:
!rm -r augmented_data

In [197]:
%cd visualization

[Errno 2] No such file or directory: 'visualization'
/home/ubuntu/workspace/visualization
